In [ ]:
from helper_functions import *

In [ ]:
# Read from saved Twitter data, preprocess, and extract metadata
# The data is created by twitter_gatherer.py
# see https://github.com/r0zetta/twitter_gather for details
texts = []
sns = []
ids = []
count = 0
print("Reading raw twitter objects...")
filename = "data/raw.json"
with open(filename, "r") as f:
    for line in f:
        count += 1
        if count % 1000000 == 0:
            print("Seen: " + str(count) + " captured: " + str(len(texts)))
        status = json.loads(line.strip())
        text = status["text"]
        sn = status["user"]["screen_name"]
        id_str = status["id_str"]
        lang = status["lang"]
        # Omit non-EN tweets
        if lang != "en":
            continue
        # Omit retweets
        if "retweeted_status" in status and status["retweeted_status"] is not None:
            continue
        if len(text) > 20 and "…" not in text:
            text = text.lower()
            text = re.sub(r'(@\S+)', '', text)
            text = re.sub(r'(http\S+)', '', text)
            text = text.strip()
            texts.append(text)
            sns.append(sn)
            ids.append(id_str)
print("Found " + str(len(texts)) + " unique texts.")

In [ ]:
# Tokenize extracted data and save all files for use later
all_sns = []
all_ids = []
all_tokens = []
all_tokens_sw = []
all_texts = []
seen = set()
min_tokens = 5
for i, text in enumerate(texts):
    if text in seen:
        continue
    seen.add(text)
    sn = sns[i]
    id_str = ids[i]
    if i % 100000 == 0:
        print(str(i) + "/" + str(len(texts)) + "/" + str(len(all_tokens)))
    clean_tokens, clean_tokens_with_sw = custom_tokenize(text)
    # Keep tweets containing a minimum number of tokens after
    # preprocessing and tokenization
    if len(clean_tokens_with_sw) >= min_tokens:
        all_sns.append(sn)
        all_ids.append(id_str)
        all_tokens.append(clean_tokens)
        all_tokens_sw.append(clean_tokens_with_sw)
        all_texts.append(text)
print(len(all_texts))
if not os.path.exists("preprocessed"):
    os.makedirs("preprocessed")
save_json(all_sns, "preprocessed/sns.json")
save_json(all_ids, "preprocessed/ids.json")
save_json(all_tokens, "preprocessed/tokens.json")
save_json(all_tokens_sw, "preprocessed/tokens_sw.json")
save_json(all_texts, "preprocessed/texts.json")
print("Done. Now execute encode_all_vectors.ipynb")